# 12. TEXT-TO-SQL

1. [x] введение
2. [x] UniSAr
3. [x] упражнение
4. [x] ссылки

# 1. Введение

Задача text-to-SQL --- преобразование вопроса на естественном языке в SQL-запрос.

> Пример
>
> Текст: `Выбрать студентов, получающих стипендию, равную 150.`
>
> SQL-запрос: `SELECT name FROM student WHERE amount=150;`

Можно использовать модели для задачи code generation, например, модели семейства BART.
Какие возникаются сложности?
Для построения корректных запросов необходима информация о базе данных (схема, логическая структура и т.д.).

В итоге, как правило, возникают специальные архитектурные решения --- нужно, чтобы генеративная модель использовала информацию о базе данных.

Подходы делятся условно на *invasive* и *non-invasive*:
- *invasive* --- языковая модель существенно интегрирована в решение
- *non-invasive* --- задача генерации выделена в отдельную подзадачу, генеративную модель можно заменить легко заменить

Нам потребуются следующие понятия:
- *multi-table* --- способность модели обрабатывать несколько таблиц
- *multi-domain* --- способность модели обрабатывать базы данных из разных предметных областей
- *multi-turn* --- способность модели работать с уточнением запросов: модель учитывает историю взаимодействия с пользователем

# 2. UniSAr

## 2.1 Статья

[Dou et al - UniSAr: A Unified Structure-Aware Autoregressive Language Model for Text-to-SQL](https://arxiv.org/abs/2203.07781)

**Uni**fied **S**tructure-**A**ware Autoreg**r**essive Language Model
- non-invasive
- multi-table
- multi-domain
- multi-turn

![](./res/12_unisar_paper.png)

## 2.2 Подход


Общая схема:

![](./res/12_unisar_overview.png)

Использованные решения:
- файнтюненный [BART-Large](https://huggingface.co/dreamerdeo/mark-bart/tree/main)
- структурные метки (structure mark) --- для кодирования структурной информации (например, схема базы данных, контекст разговора, связи)
- декодирование с ограничениями (constrained decoding) --- фильтрация недопустимых токенов
- SQL-завершение (SQL completion) ---добавление необходимых `JOIN` на основе предсказанного неполного SQL и схемы базы данных

## 2.3 Предобученная языковая модель

Используется [BART-Large](https://huggingface.co/dreamerdeo/mark-bart/tree/main) (Bidirectional Auto-Regressive Transformers) --- полный трансформер.

Вход модели:
- информация о базе данных
- история
- запрос на естественном языке

Информация о базе данных кодируется в виде последовательности токенов:
$$T = [TABLE], t_1, \ldots, t_N, [COLUMN], c_1, \ldots, c_N,$$
где $[TABLE]$ и $[COLUMN]$ --- специальные токены, указывающие область заголовков таблиц и имен столбцов соответственно.

Информация о базе данных и текстовая информация конкатенируется.
Кроме того, добавляется история диалога (multi-turn).

![](./res/12_unisar_input.png)

## 2.4 Структурные метки

Идея: давайте закодируем всё необходимую информацию о базе данных в виде последовательности токенов.

Какая информация нас интересует?
- свойства схемы (schema property) для расширения семантической информации схемы;
- структура базы данных (database structure) для агрегирования информации от соседей по схеме;
- контекст взаимодействия (discourse structure)

### 2.4.1 Свойства схемы (schema property)

В естественном языке много синонимов. Одно и то же можно сказать по-разному.
При этом, SQL-запрос должен опираться на схему БД.

Какая информация может быть полезной?
1. такая информация из схемы базы данных, как первичный ключ или тип столбца (`INT`, `STRING` или `DATE`)
2. информацию о связывании на основе названий между текстовым вопросом и схемой
3. информация о связывании на основе значений (содержимого база данных) --- больше информации о столбце

В целом получить свойство схемы относительно просто.

Например, часть информации (п1) содержится в определении базы данных.
Информацию о связывании можно получить с помощью $n$-грамм.

Для более точного указания найденного соответствия,
используются префиксы `Partial` или `Exact` перед `Match`.

> Пример.
> ![](./res/12_unisar_input_column.png)
> Столбец `Player_Id` частично перекрывается c токеном `Player`, поэтому добавляется `Partial-Match` перед столбцом `Player_Id`.

Для нахождения таких соответствий (п2) используется лемматизация и алгоритм нечёткого сравнения строк.

Для информации о связывании на основе значений (п3):
1. данные приводятся к единому виду (например, унифицируется представление дат)
2. затем данные сопоставляются с токеном из вопроса.

Вся полученная информация (пп1--3) добавляется в виде префиксов.

> Пример.
> `Partial-Match & Primary-Key & Integer Ranking.Player_id`
> `Ranking.Player_id` --- это столбец.
Префикс `Partial-Match & Primary-Key & Integer` является структурной меткой, которая выражает следующие свойства схемы:
> 1. `Partial-Match` указывает, что `Ranking.Player_id` частично соответствует вопросу
> 2. `Primary-Key` и `Integer` являются свойствами столбца


???

![](./res/12_unisar_input_table.png)


### 2.4.2  Структура базы данных (database structure)

Структура базы данных может улучшить представление схемы за счет агрегирования информации с соседних сущностей.
Структура базы данных включает
- связи между столбцами и таблицами (например, `Id` для `Matches`);
- связи внешнего ключа между столбцами (например, `WinnerId` ссылается на `PlayerId`);
- связи между таблицами (например, `Matches` ссылается на `Ranking`).

![](./res/12_unisar_database.png)

Для установления связей между таблицами добавляются ссылки для столбцов (например, `Matches.Id`), отношение внешних ключей.

Остаётся добавить отношение между таблицами.
Это делается по шаблону *schema1 links to schema2* и добавляется ко входу модели.
Информация об отношениях между таблицами влияет на корректность генерации части SQL-запроса, относящегося к `FROM`.

![](./res/12_unisar_input_database.png)


### 2.4.3 Discourse structure

Для обеспечения свойства *multi-turn* ко входу добавляется история и предыдущий ответ модели.

![](./res/12_unisar_input_discourse.png)
![](./res/12_unisar_input_history.png)

## 2.5 Декодирование с ограничениями

Выход модели должен быть корректным с точки зрения:

1. синтаксиса языка SQL
2. схемы БД

Модель BART часто выдаёт корректный SQL-код, но может использовать синонимы, которые не учитывают базу данных.
Для этого строится префиксное дерево (trie), используя которое, происходит фильтрация недопустимых токенов на уровне beam search.

![](./res/12_unisar_constrained_decoding.png)

## 2.6 SQL-завершение

Оказывается, что в сгенерированных SQL-запросах часто отсутствуют некоторые компоненты `JOIN`.
Они часто не упоминаются явным образом в вопросах на естественном языке.

![](./res/12_unisar_incomplete.png)

Таким образом, необходимо найти потенциально отсутствующие компоненты `JOIN` на основе схемы базы данных и добавить из в запрос.

![](./res/12_unisar_complete.png)

Для этого строится граф схемы, где узлы --- это таблицы или столбцы, а ребра --- связи.

![](./res/12_unisar_graph_complete.png)

Затем находим те узлы (таблицы и столбцы), которые расположены на кратчайшем путь к существующим таблицам и столбцам в незавершенном SQL запросе.

## 2.7 Evaluation

### 2.7.1 Датасеты

![](./res/12_unisar_datasets.png)

### 2.7.2 Метрики

- LX: logic form accuracy (WikiSQL, TableQA)
    - процент точного совпадения строк между сгенерированными SQL-запросами и реальными SQL-запросами
- EX: execution accuracy (WikiSQL, TableQA)
    - процент сгенерированных запросов SQL, которые приводят к правильному ответу
- EM: exact set match accuracy (Spider, DuSQL)
    - разбивается каждый SQL на несколько частей и затем части сравниваются (это помогоает понимать, что такие, например, запросы `SELECT col1, col2` и `SELECT col2, col1`, являются эквивалентными)
- QM: question match accuracy (SParC, CoSQL, Chase)
    - exact set matching score для всех вопросов
- IM: interaction match accuracy (SParC, CoSQL, Chase)
    - exact set matching score для всех взаимодействий

### 2.7.3 Результаты

![](./res/12_unisar_results_1.png)
![](./res/12_unisar_results_2.png)
![](./res/12_unisar_results_3.png)
![](./res/12_unisar_results_4.png)

### 2.7.4 Пример работы

![](./res/12_unisar_case_study.png)

## 2.8 Реализация

- github: [microsoft/ContextualSP/tree/master/unified_parser_text_to_sql](https://github.com/microsoft/ContextualSP/tree/master/unified_parser_text_to_sql)
- реализован на основе [Fairseq](https://github.com/facebookresearch/fairseq)
- обучение заняло 10 часов на четырёх V100-16G GPU:
    - BART-Large
    - задача `TRANSLATION_FROM_PRETRAINED_BART` (Fairseq)
    - около 7000 шагов (примерно 10--15 эпох) на Spider, CoSQL и SParC

![](./res/12_unisar_inference.png)

# 3. Упражнение

Провести исследование продуктов на основе технологии text-to-SQL (на что нацелен продукт, для кого и тд)

# 4. Ссылки

- https://arxiv.org/pdf/2204.00498.pdf
- https://habr.com/ru/post/480838/
- https://arxiv.org/pdf/1808.06304.pdf
- https://aclanthology.org/P19-1443.pdf
- https://github.com/taoyds/spider/tree/master/evaluation_examples